<a href="https://colab.research.google.com/github/NMashalov/Python-MIPT-education-course-2023-Spring/blob/main/Part2.Telegram/Python_Generators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Ленивые вычсиления

Ленивые (lazy) вычисления - термин пришедший из функционального программирования
Это означает, что вычисления выполняются на ходу, а не заранее.
Такой подход позволяет сократить расходы памяти


Функция стандартного пакета `sys.getsizeof` позволяет узнать расход памяти на объект в памяти.
Проверим зависит ли размер примитива `range` от его длины

In [ ]:
import sys
print(sys.getsizeof(range(10)))
print(sys.getsizeof(range(1_000)))
print(sys.getsizeof(range(1_000_000)))

48
48
48


Попробуем также с `list`

In [ ]:
print(*(sys.getsizeof(list(range(i))) for i in [10,1_000,1_000_00]))

136 8056 800056


Размер затраченной памяти начинает прирастать
Это критично в задачах, требующих работы с большими данными или выполняющими работу на протяжении долгого времени

In [ ]:
print(sum(range(1_000_000_000)))

499999999500000000


In [ ]:
print(sum(list(range(1_000_000_000))))

Процесс не выполнится и это нормально :) Нам не хватило памяти для хранения такого большого листа.  А справится ли специальна библиотека для вычислений numpy?

In [ ]:
import numpy
x = numpy.arange(1_000_000_000)

Да, массив поместился в память.
Представление целого числа в numpy примерно в 10 раз легче :)

In [ ]:
import sys
import numpy
print(sys.getsizeof(list(range(1_000_000))))
print(sys.getsizeof(numpy.arange(1_000_00)))

8000056
800112


Попробуем просуммировать встроенным методом

In [ ]:
x.sum()

Не получилось :(
Дело во внутренних оптимизациях numpy. Если использовать обычный sum все получится

In [ ]:
import numpy
sum(numpy.arange(1_000_000_000))

499999999500000000

Расчет получился, но был очень долгий. Numpy при каждом обращении к массиву выполняет очень много работы по обработке исключений и вызове исходного кода на c.

Но почему вызовов было много 🤔 Дело в том, что функция sum в python устроена рекурентно:



In [ ]:
list_to_sum = range(11)
# последовательный обходчик
walker = iter(list_to_sum)
recurrent_unit = next(walker)
print('Cнаружи цикла:', recurrent_unit)
for x in walker:
    print('Внутри цикла:', x)
    recurrent_unit += x
print(recurrent_unit)

Cнаружи цикла: 0
Внутри цикла: 1
Внутри цикла: 2
Внутри цикла: 3
Внутри цикла: 4
Внутри цикла: 5
Внутри цикла: 6
Внутри цикла: 7
Внутри цикла: 8
Внутри цикла: 9
Внутри цикла: 10
55


`Iter` - это указатель. Он умеет три вещи:
- вызывать у структуры метод-дескриптор `__iter__`
- возвращать при вызове функции `next` элемент последовательности
- вызывать метод последовательности `__next__` для понимания какой элемент следующий

Поэтому размер iter не зависит от размера последовательности и даже размера ее элементов


In [ ]:
import sys
print(sys.getsizeof(iter(range(10))))
print(sys.getsizeof(iter(range(1_000_000))))
print(sys.getsizeof(iter([list(range(1_000_000)) for i in range(10)])))

48
48
48


Обманем `iter`. Заставим его шагать по структуре не похожую на последовательность

In [ ]:
class Trick:
    def __iter__(self):
        print('Обманули :)')
        return self
    def __next__(self):
        return 'Нет тут ничего'


t = iter(Trick())
next(t),next(t)

Обманули :)


('Нет тут ничего', 'Нет тут ничего')

Для работы с рекуррентными структурами также удобно использовать `functools.reduce`. Например, таким образом можно эффективно вычислить факториал

In [ ]:
import functools
print(functools.reduce(lambda x,y: x*y, range(1,25)))

620448401733239439360000

### 📝 Задача

1. Сложите 1_000_000_000 равномерно распределенных чисел.

In [ ]:
# для задания равномерно распределенного числа можно использовать random
from random import random

random()

0.9296776503022031

🦹 Вредные советы

Я предложу плохую реализацию, как опорный пример. Покритикуйте подход, можно ли его улучшить?

Решение через трюк с итератором

In [15]:
import random
class Trick:
    def __init__(self,limit:int):
        self.limit = limit
    def __iter__(self):
        self.counter = 0
        return self
    def __next__(self):
        self.counter += 1
        if self.counter <= self.limit:
            return random.random()
        else:
            raise StopIteration

sum(Trick(1_000_000_000))

499994098.1147857



2. Придумайте как с помощью iter обходить только четные элементы в последовательности

In [ ]:
x = range(6)
result = # Ваша магия 🔮 с iter() и x
assert next(result)==1
assert next(result)==3
assert next(result)==5

🦹 Вредные советы

In [25]:
import typing as tp
class EvenIterator:
    def __init__(self, sequence: tp.Iterable):
        self.sequence = list(sequence)[1::2]
    def __iter__(self):
        return iter(self.sequence)

x = range(6)
result =iter(EvenIterator(x))
assert next(result)==1
assert next(result)==3
assert next(result)==5

**Вывод**: по возможности используйте ленивые вычисления. Они значительно экономят расход памяти

### Генераторы
Это правило, которое задает последовательность на каждом ее шаге.

📐Понятие пришло из теории рядов. Так, например, в теории групп Ли аргумент функции $\exp(w)$ называют генератором, потому что на самом деле $w$ аргумент ряда $\sum_{i=1}^\infty \frac{w^i}{n!}$

Генераторы в python можно задать двумя простыми способами

In [ ]:
import random
# через скобки
generator_1 = (random.random() for i in range(10))
# через ключевое слово yield в функции
def generator_2():
    for i in range(10):
        yield random.random()

Вызвов генератор выполняется с помощью итератора

In [ ]:
for item in generator_1:
    print(item)
print(*generator_2())

0.6353597912927114
0.18085771759577274
0.48907213214096557
0.9511970481915663
0.33085704422149453
0.8610810454924469
0.04717197899797365
0.03157459750462488
0.5979285268247055
0.9524130248180428
0.5182904703764644 0.2749009307039585 0.6247895840259199 0.5365265245578852 0.6112135243809073 0.056363616594985344 0.1758741099282567 0.38201482424296285 0.308585552756096 0.1474230751110105


Обратите внимание, что генератор заданного через `()` нельзя вызвать два раза, а через функцию можно

In [ ]:
for item in generator_1:
    print(item)
print(*generator_2())

0.7434596839216792 0.9523808881335253 0.7408340751405318 0.5311508979576381 0.6433568850557354 0.49790071836777516 0.607993575028079 0.09296883620622576 0.25710656555403943 0.5423733459315748


Дело в том, что функция с конструкцией `yield` возвращает `generator`, а не сами числа

In [ ]:
type(generator_2), type(generator_2()),type(next(generator_2())),

(function, generator, float)

Генераторы очень удобно использовать для чтения из файлов

Создадим небольшой файл, содержащий числа от 1 до 20 на каждой новой строке

In [ ]:
with open('example.txt','w') as f:
    for i in range(20):
        f.write(f'{i}\n')

Jupyter notebook позволяет вызывать команды bash из ячейки. Для этого используем специальный символ `!`.

Для вывода оодержимого файла в bash используется команда `cat`

In [ ]:
!cat example.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


Попробуем написать функцию, которая возвращает файл для чтения

In [ ]:
def bad_reader():
    with open('example.txt','r') as f:
        return f

In [ ]:
f = bad_reader()
f.read()

ValueError: I/O operation on closed file.

Ошибка означает, что соединение с файлом при начале его чтения уже было закрыто. Подключение было прервано контекстным менеджером `with` после того, как функция `bad_reader` завершила свою работу

In [ ]:
def good_reader():
    with open('example.txt','r') as f:
         for line in f:
            yield line

In [ ]:
list(good_reader())

['0\n',
 '1\n',
 '2\n',
 '3\n',
 '4\n',
 '5\n',
 '6\n',
 '7\n',
 '8\n',
 '9\n',
 '10\n',
 '11\n',
 '12\n',
 '13\n',
 '14\n',
 '15\n',
 '16\n',
 '17\n',
 '18\n',
 '19\n']

Все получилось :) В данном случае генератор with понимает, что отключаться от файла нужно будет при полном прочтении файла

Также генераторы удобны в обработке потоков

Смешаем поток возрастающих и случайных чисел. Например, `range` и `random`

In [ ]:
def range_generator():
    yield from range(10)

def random_generator():
    return (random.random() for _ in range(10))

def mix_generator(generator_1, generator_2):
    for value_from_gen_1, value_from_gen_2 in zip(generator_1,generator_2):
        yield value_from_gen_1
        yield value_from_gen_2

print(*mix_generator(range_generator(),random_generator()))

0 0.16483825214953907 1 0.7512267521616811 2 0.8078975381798265 3 0.9727118761043366 4 0.9967076990911371 5 0.8769332532338354 6 0.7377729685934948 7 0.5332234446968888 8 0.3722443312898651 9 0.7346960919612103


Конструкция `yield from` позволяет создавать генератор из другого генератора или из последовательности

Обработчики можно композировать. Например сначала добавив число, а потом на него домножив

In [ ]:
import typing as tp
def add_stream(gen: tp.Iterable):
    for i in gen:
        yield i + 10

def mult_stream(gen: tp.Iterable):
    for i in gen:
        yield i * 20

In [ ]:
gen = range(10)
print(*mult_stream(add_stream(gen)))

200 220 240 260 280 300 320 340 360 380


Обратим внимание, что обработка в данной реализации выполняется последовательно.
Выполнение следующей итерации начинается только после завершения предыдущей.

In [ ]:
import time
def sleep_stream(gen: tp.Iterable):
    for i in gen:
        print('Подождали')
        time.sleep(0.5)
        yield i

def print_stream(gen: tp.Iterable):
    for i in gen:
        print(i)
        yield i

gen = range(10)
print(*print_stream(sleep_stream(gen)))

Подождали
0
Подождали
1
Подождали
2
Подождали
3
Подождали
4
Подождали
5
Подождали
6
Подождали
7
Подождали
8
Подождали
9
0 1 2 3 4 5 6 7 8 9


Поскольку работа идет поэлементно, то вычисления можно прооводить даже с очень большими последовательностями

In [ ]:
gen = (i*5 for i in range(100_000_000))
print(sum(mult_stream(add_stream(gen))))

500000015000000000


### 📝 Задача
1. Реализовать `range` с типом возвращаемого числа не `int`, а `str`


Ваше решение

🦹 Вредные советы


In [ ]:
def string_range(start=0, end =5, step=1):
    trajectory = [str(i) for i in range(start,end,step)]
    yield from trajectory

assert tuple(string_range())==("0","1","2","3","4")

2. Реализуйте генератор чисел Фибоначи до числа `n`

🦹 Вредные советы


In [2]:

def fib(n=5):
    fib_prev,fib=1,1
    yield fib_prev
    yield fib
    yield from (fib := fib_prev + (fib_prev := fib) for _ in range(n-2))

list(fib(5))

[1, 1, 2, 3, 5]

### Бонус 🎆 Реализация асинхронности через генераторы

В примере
```python
import time
def sleep_stream(gen: tp.Iterable):
    for i in gen:
        print('Подождали')
        time.sleep(0.3)
        yield i
def print_stream(gen: tp.Iterable):
    for i in gen:
        print(i)
        yield i
gen = range(10)
print(*sleep_stream(print_stream(gen)))
```
мы почти все время ждали исполнения оператора `sleep_stream` прежде чем приступить к следующей итерации. Как можно было бы начинать на следующую задачу во время сна?

А что если вместо того, чтобы ждать, возвращать само ождиание 🤔

Реализуем класс, который принимает значение `value` и домножает его на 10 за промежуток времени `wait_time`.  

При инициализации класса он сразу начинает вычисление, представленное генератором. Если мы еще не успели посчитать, генератор ничего не возвращает - `None`, если вычисления уже выполнены, то возвращается результат вычисления `value*10`

In [ ]:
import time

class AwaitableMult:
    def __init__(self,value,wait_time):
        self.wait_time = wait_time
        self.value = value
        # и сразу запускаем
        self.result = self.wait_value()
        next(self.result)

    def wait_value(self):
        init_time = time.time()
        now =  time.time()
        while now - init_time < self.wait_time: #ждем следующего вызова для проверки времени
            yield
            now = time.time()
        print(f'Результат {self.value* 10}')
        yield self.value * 10

Для работы с таким классом мы реализуем опрашивающий цикл.

Каждую десятую секунды мы будем спрашивать у генератора есть ли у него результат. Если его еще нет, то подождем ответа.
Функция опроса будет записывать результаты выполнения в лист.

In [ ]:
class Poll:
    def __init__(self, awaitables: list[AwaitableMult]):
        self.answers = []
        self.awaitables = awaitables

    def poll(self,awaitable):
        # опрашиваем генератор
        result = next(awaitable.result)
        if result is None:
            return True
        else:
            self.answers.append(result)
            return False

    def cycle(self):
        while len(self.awaitables) > 0:
            self.awaitables = [awaitable for awaitable in self.awaitables if self.poll(awaitable)]
            # каждую 0.1 секунды опрашиваем закончилось ли ожидание в задаче
            time.sleep(0.1)
        return self.answers


In [ ]:
from os import truncate
import time
import random
import typing as tp

# последовательно мы бы ожидали 21 секунду
# но в итоге прождем всего 5
awaitables= [AwaitableMult(i,i) for i in range(1,6)]
answers = Poll(awaitables).cycle()
print(answers)

Результат 10
Результат 20
Результат 30
Результат 40
Результат 50
[10, 20, 30, 40, 50]


Обратите внимание, что запуск выполняется сразу после создания класса `AwaitableMult`.
Чтобы проверить это свойство, подождем три секунды перед запуском

In [ ]:
awaitables= [AwaitableMult(i,i) for i in range(1,6)]
time.sleep(3)
answers = Poll(awaitables).cycle()
print(answers)

Результат 10
Результат 20
Результат 30
Результат 40
Результат 50
[10, 20, 30, 40, 50]


**Вывод**: Такой подход называется асинхроным или конкурентным. Главное преимущество подхода заключается в совместном, а не последовательном ожидании. Это актуально, когда мы собираем информацию из большого числа источников.

Также мы научились реализовывать примитивы асинхроности используя генераторы.
В программировании они называются:
- `event_loop` - агент, опрашивающий в промежутки времени статус исполнения задачи
- `coroutine` - функция стартующая задачу иумеющая сказать завершилась ли она или нет

Стандартный пакет `asyncio` развивает подход набором удобных инструментов, позволяет опрашивать с разным приоритетом и сроком, обработывает исключения.

